<a href="https://colab.research.google.com/github/nagasatvika/semantic-similarity/blob/main/bonus_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from datasets import load_dataset

dataset = load_dataset("paws", "labeled_final")

In [9]:
from datasets import load_dataset

dataset1 = load_dataset("PiC/phrase_similarity")

Fine-Tuning Pre-Trained BERT Model

Note: Used chatgpt for the code as this concept is new for me

In [10]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the PiC/phrase_similarity dataset
pic_dataset = load_dataset("PiC/phrase_similarity")

# Extract the first two phrases from the PiC/phrase_similarity dataset
pic_phrase1 = pic_dataset['test'][0]['phrase1']
pic_phrase2 = pic_dataset['test'][0]['phrase2']

# Load a pre-trained model and tokenizer
model_name = "textattack/bert-base-uncased-MNLI"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize the phrases from the PiC/phrase_similarity dataset
pic_tokenized_input = tokenizer(pic_phrase1, pic_phrase2, return_tensors='pt', padding=True, truncation=True)

# Compute the similarity score for the PiC/phrase_similarity dataset
with torch.no_grad():
    pic_output = model(**pic_tokenized_input)
pic_similarity_score = torch.softmax(pic_output.logits, dim=1)[0][1].item()  # Take the score for "entailment" class

print("PiC Dataset")
print("Phrase 1:", pic_phrase1)
print("Phrase 2:", pic_phrase2)
print("Similarity Score:", pic_similarity_score)

# Load the PAWS dataset
paws_dataset = load_dataset("paws", "labeled_final")

# Extract the first two phrases from the PAWS dataset
paws_phrase1 = paws_dataset['test'][0]['sentence1']
paws_phrase2 = paws_dataset['test'][0]['sentence2']

# Tokenize the phrases from the PAWS dataset
paws_tokenized_input = tokenizer(paws_phrase1, paws_phrase2, return_tensors='pt', padding=True, truncation=True)

# Compute the similarity score for the PAWS dataset
with torch.no_grad():
    paws_output = model(**paws_tokenized_input)
paws_similarity_score = torch.softmax(paws_output.logits, dim=1)[0][1].item()  # Take the score for "entailment" class

print("\nPAWS Dataset")
print("Sentence 1:", paws_phrase1)
print("Sentence 2:", paws_phrase2)
print("Similarity Score:", paws_similarity_score)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

PiC Dataset
Phrase 1: air position
Phrase 2: posture while jumping
Similarity Score: 0.08406110852956772

PAWS Dataset
Sentence 1: This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic .
Sentence 2: This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic .
Similarity Score: 0.9858448505401611


#Using commercial LLM APIs i.e (CHAT-GPT)

#Sentence similarity

In [11]:
!pip install datasets scikit-learn


#ZERO SHOT LEARNING

In [12]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Load the dataset
dataset = load_dataset("paws", "labeled_final", split="test[:100]")  # Using a subset for efficiency
# Extract the sentences
sentences1 = dataset['sentence1']
sentences2 = dataset['sentence2']
# Combine all sentences for TF-IDF fitting
all_sentences = sentences1 + sentences2
# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()
# Fit and transform the sentences
tfidf_matrix = vectorizer.fit_transform(all_sentences)
# Split the transformed matrix into two parts for each sentence pair
tfidf_sentences1 = tfidf_matrix[:len(sentences1)]
tfidf_sentences2 = tfidf_matrix[len(sentences1):]
# Calculate cosine similarity for each pair of sentences
similarity_scores = cosine_similarity(tfidf_sentences1, tfidf_sentences2)
# Create a list of similarity scores for each sentence pair
similarity_scores_diagonal = similarity_scores.diagonal()
# Output the first few similarity scores as a sample
print(similarity_scores_diagonal[:10])

[0.86965731 0.98411651 0.93038253 0.8584274  1.         0.95025992
 0.91144189 0.98411651 0.92410881 0.98314708]


#Few Shot Setting

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

def preprocess(sentence):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(sentence.lower())
    filtered_sentence = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_sentence)

def cosine_similarity_score(sentence1, sentence2):
    preprocessed_sentence1 = preprocess(sentence1)
    preprocessed_sentence2 = preprocess(sentence2)

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([preprocessed_sentence1, preprocessed_sentence2])

    similarity_score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

    return similarity_score
# Test cases
sentences = [
    ("This was a series of nested angular standards, so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic.",
     "This was a series of nested polar scales, so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic."),
    ("His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America.",
     "His father emigrated to America in 1868, but returned when his wife became ill and before the rest of the family could go to Missouri."),
    ("In January 2011, the Deputy Secretary General of FIBA Asia, Hagop Khajirian, inspected the venue together with SBP-President Manuel V. Pangilinan.",
     "In January 2011, FIBA Asia deputy secretary general Hagop Khajirian along with SBP president Manuel V. Pangilinan inspected the venue.")
]

for idx, (sentence1, sentence2) in enumerate(sentences):
    similarity_score = cosine_similarity_score(sentence1, sentence2)
    print(f"Similarity score between sentences {idx + 1}: {similarity_score}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Similarity score between sentences 1: 0.7523197619890015
Similarity score between sentences 2: 0.9317157650164152
Similarity score between sentences 3: 0.8836351388995085


#Phrase Similarity

Zero Shot Setting

In [14]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
# Load the dataset
dataset = load_dataset("PiC/phrase_similarity")
 # Extract the first two phrases
phrase1 = dataset['test'][0]['phrase1']
phrase2 = dataset['test'][0]['phrase2']
# Load a pre-trained model and tokenizer
model_name = "textattack/bert-base-uncased-MNLI"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# Tokenize the phrases
tokenized_input = tokenizer(phrase1, phrase2, return_tensors='pt', padding=True, truncation=True)
# Compute the similarity score
with torch.no_grad():
    output = model(**tokenized_input)
similarity_score = torch.softmax(output.logits, dim=1)[0][1].item()  # Take the score for "entailment" class
print("Phrase 1:", phrase1)
print("Phrase 2:", phrase2)
print("Similarity Score:", similarity_score)

Phrase 1: air position
Phrase 2: posture while jumping
Similarity Score: 0.08406110852956772


Few Shot Setting

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(phrase1, phrase2):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([phrase1, phrase2])
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix[0, 1]

phrase1 = "air position"
phrase2 = "posture while jumping"
phrase3 = "associated track"
phrase4 = "correlating music single" 		#Score:0.4104
phrase5 = "whole parts"
phrase6 = "extended sections" 		#Score:0.3237
similarity_score1 = calculate_similarity(phrase1, phrase2)
similarity_score2 = calculate_similarity(phrase3, phrase4)
similarity_score3 = calculate_similarity(phrase5, phrase6)
print(f"Similarity score between '{phrase1}' and '{phrase2}': {similarity_score1}")
print(f"Similarity score between '{phrase3}' and '{phrase4}': {similarity_score2}")
print(f"Similarity score between '{phrase5}' and '{phrase6}': {similarity_score3}")

Similarity score between 'air position' and 'posture while jumping': 0.0
Similarity score between 'associated track' and 'correlating music single': 0.0
Similarity score between 'whole parts' and 'extended sections': 0.0


Using open source LLMs/APIs i.e (Mistral AI)

In [2]:
!pip install sentence-transformers
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

#Sentence Similarity

#Zero Shot Setting

In [16]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the dataset
dataset = load_dataset("paws", "labeled_final")
# Load a pre-trained model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
# Get a pair of sentences from the dataset
for i in range(5):
  sentence1 = dataset['test'][i]['sentence1']
  sentence2 = dataset['test'][i]['sentence2']
# Calculate the embeddings for the sentences
  embedding1 = model.encode(sentence1)
  embedding2 = model.encode(sentence2)
# Calculate the cosine similarity between the embeddings
  similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

  print(f"Sentence 1: {sentence1}")
  print(f"Sentence 2: {sentence2}")
  print(f"Similarity score: {similarity_score}\n")


Sentence 1: This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic .
Sentence 2: This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic .
Similarity score: 0.9322909116744995

Sentence 1: His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America .
Sentence 2: His father emigrated to America in 1868 , but returned when his wife became ill and before the rest of the family could go to Missouri .
Similarity score: 0.9918646812438965

Sentence 1: In January 2011 , the Deputy Secretary General of FIBA Asia , Hagop Khajirian , inspected the venue together with SBP - President Manuel V. Pangilinan .
Sentence 2: In January 2011 , FIBA Asia deputy secretary general Hagop Khajirian along with SBP presid

Few Shot Setting

In [4]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the dataset
dataset = load_dataset("paws", "labeled_final")

# Load a pre-trained model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Calculate the similarity for the first 5 pairs of sentences
for i in range(5):
    sentence1 = dataset['test'][i]['sentence1']
    sentence2 = dataset['test'][i]['sentence2']

    # Calculate the embeddings for the sentences
    embedding1 = model.encode(sentence1)
    embedding2 = model.encode(sentence2)

    # Calculate the cosine similarity between the embeddings
    similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

    print(f"Sentence 1: {sentence1}")
    print(f"Sentence 2: {sentence2}")
    print(f"Similarity score: {similarity_score}\n")


Sentence 1: This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic .
Sentence 2: This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic .
Similarity score: 0.9322909116744995

Sentence 1: His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America .
Sentence 2: His father emigrated to America in 1868 , but returned when his wife became ill and before the rest of the family could go to Missouri .
Similarity score: 0.9918646812438965

Sentence 1: In January 2011 , the Deputy Secretary General of FIBA Asia , Hagop Khajirian , inspected the venue together with SBP - President Manuel V. Pangilinan .
Sentence 2: In January 2011 , FIBA Asia deputy secretary general Hagop Khajirian along with SBP presid

Zero Shot Setting

In [5]:
from datasets import load_dataset
import sentence_transformers

# Load the dataset
dataset1 = load_dataset("PiC/phrase_similarity")

# Get the two phrases you want to compare
phrase1 = dataset1['test'][0]['phrase1']
phrase2 = dataset1['test'][0]['phrase2']

# Initialize a Sentence Transformer model
model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')

# Convert the phrases to embeddings
embedding1 = model.encode(phrase1)
embedding2 = model.encode(phrase2)

# Calculate the cosine similarity between the embeddings
similarity_score = sentence_transformers.util.cos_sim(embedding1, embedding2)

print(f"The similarity score between '{phrase1}', and '{phrase2}' is '{similarity_score}' ")


Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

The similarity score between 'air position', and 'posture while jumping' is 'tensor([[0.5256]])' 


In [6]:
from datasets import load_dataset
import sentence_transformers

# Load the dataset
dataset1 = load_dataset("PiC/phrase_similarity")

# Get the two phrases you want to compare
phrase1 = dataset1['test'][0]['phrase1']
phrase2 = dataset1['test'][0]['phrase2']

# Initialize a Sentence Transformer model
model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')

# Convert the phrases to embeddings
embedding1 = model.encode(phrase1)
embedding2 = model.encode(phrase2)

# Calculate the cosine similarity between the embeddings
similarity_score = sentence_transformers.util.cos_sim(embedding1, embedding2)

# Convert the tensor to a Python float
#similarity_score = similarity_score.item().to('cpu')

# Print the result and compare it with the given similarity scores
print(f"The similarity score between '{phrase1}' and '{phrase2}' is {similarity_score}")

given_scores = {
    ("air position", "posture while jumping"): 0.5256,
    ("associated track", "correlating music single"): 0.4104,
    ("whole parts", "extended sections"): 0.3237,
    ("wide set", "spacious collection"): 0.4795,
    ("full protection", "complete defense"): 0.5793,
}

if (phrase1, phrase2) in given_scores:
    given_score = given_scores[(phrase1, phrase2)]
    print(f"The given similarity score is {given_score}")
    print(f"The obtained similarity score is {similarity_score}")



The similarity score between 'air position' and 'posture while jumping' is tensor([[0.5256]])
The given similarity score is 0.5256
The obtained similarity score is tensor([[0.5256]])


Few Shot Setting

In [7]:
from datasets import load_dataset
import sentence_transformers

# Load the dataset
dataset1 = load_dataset("PiC/phrase_similarity")

# Get the two phrases you want to compare
phrase1 = dataset1['test'][0]['phrase1']
phrase2 = dataset1['test'][0]['phrase2']

# Initialize a Sentence Transformer model
model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')

# Convert the phrases to embeddings
embedding1 = model.encode(phrase1)
embedding2 = model.encode(phrase2)

# Calculate the cosine similarity between the embeddings
similarity_score = sentence_transformers.util.cos_sim(embedding1, embedding2)

# Convert the tensor to a Python float
#similarity_score = similarity_score.item().to('cpu')

# Print the result and compare it with the given similarity scores
print(f"The similarity score between '{phrase1}' and '{phrase2}' is {similarity_score}")

given_scores = {
    ("air position", "posture while jumping"): 0.5256,
    ("associated track", "correlating music single"): 0.4104,
    ("whole parts", "extended sections"): 0.3237,
    ("wide set", "spacious collection"): 0.4795,
    ("full protection", "complete defense"): 0.5793,
}

if (phrase1, phrase2) in given_scores:
    given_score = given_scores[(phrase1, phrase2)]
    print(f"The given similarity score is {given_score:.4f}")
    print(f"The difference between the two scores is {abs(similarity_score - given_score)}")


The similarity score between 'air position' and 'posture while jumping' is tensor([[0.5256]])
The given similarity score is 0.5256
The difference between the two scores is tensor([[4.4346e-05]])
